# Tutorial: Deadline-Aware Processing Patterns

**Category**: Concurrency  
**Difficulty**: Intermediate  
**Time**: 20 minutes

## Overview: Why Deadline Patterns Matter

In production systems, you often need to process work within fixed time budgets:

- **ETL pipelines** must complete before the next data refresh
- **Batch notifications** need to send before daily cutoff times
- **Background jobs** must finish within maintenance windows
- **API handlers** must respect SLA timeouts

The challenge: process **as much as possible** before the deadline, return **partial results** gracefully, avoid **wasted work** on tasks that won't complete.

**This tutorial covers**:
1. Sequential deadline-aware processing (single loop, check time before each task)
2. Parallel worker pool pattern (multiple workers, shared queue, sentinel shutdown)
3. When to use each pattern
4. Production-ready copypaste code

**Key lionherd APIs**:
- `move_on_at(deadline)` - Silent cancellation at absolute time
- `effective_deadline()` - Query remaining time from ambient scopes
- `current_time()` - Monotonic clock for deadline calculations
- `Queue.with_maxsize(n)` - Bounded FIFO queue with backpressure

## Prerequisites

**Required**:
```bash
pip install lionherd-core  # >=0.1.0
```

**Prior Knowledge**:
- Python async/await fundamentals
- Basic understanding of queues and batch processing

**Related Resources**:
- [API Reference: Cancellation Utilities](../../docs/api/libs/concurrency/cancel.md)
- [API Reference: Primitives](../../docs/api/libs/concurrency/primitives.md)

In [1]:
# Standard library
from collections.abc import Callable
from dataclasses import dataclass
from typing import Any

# lionherd-core
from lionherd_core.libs.concurrency import (
    Queue,
    create_task_group,
    current_time,
    effective_deadline,
    move_on_at,
    sleep,
)

## API Overview

### `move_on_at(deadline)` - Graceful Degradation

Silently cancels at absolute deadline. Returns partial results without exceptions.

```python
deadline = current_time() + 5.0
with move_on_at(deadline) as scope:
    results = await process_tasks()
if scope.cancel_called:
    print("Deadline reached")
```

### `effective_deadline()` - Query Remaining Time

Returns nearest deadline from ambient scopes (`float` or `None`).

```python
if effective_deadline() and (effective_deadline() - current_time()) < 0.1:
    print("Skip task, insufficient time")
```

### `current_time()` - Monotonic Clock

For measuring intervals and calculating deadlines.

```python
start = current_time()
await work()
elapsed = current_time() - start
```

## Pattern 1: Sequential Deadline-Aware Processing

**When to use**: Tasks must run sequentially (dependencies, rate limits, single resource access)

**Key insight**: Check remaining time **before** starting each task to avoid wasting work on tasks you can't finish.

**Pattern**:
```
deadline = current_time() + 30.0
with move_on_at(deadline):
    for task in tasks:
        if (deadline - current_time()) < min_time:
            break  # Not enough time
        result = await task()
```

In [2]:
# Sequential deadline-aware processing


@dataclass
class Task:
    """Simple task representation."""

    id: str
    work: Callable[[], Any]  # Async callable


@dataclass
class Result:
    """Simple result representation."""

    task_id: str
    success: bool
    data: Any = None
    error: str | None = None


async def process_sequential_with_deadline(
    tasks: list[Task],
    deadline: float,
    min_time: float = 0.01,
) -> tuple[list[Result], int]:
    """Process tasks sequentially until deadline.

    Args:
        tasks: List of tasks to process
        deadline: Absolute deadline (from current_time())
        min_time: Skip task if less than this time remaining

    Returns:
        (results, skipped_count)
    """
    results = []
    skipped = 0

    with move_on_at(deadline) as scope:
        for task in tasks:
            # Check if we have enough time
            remaining = deadline - current_time()
            if remaining < min_time:
                skipped = len(tasks) - len(results)
                break

            try:
                data = await task.work()
                results.append(Result(task_id=task.id, success=True, data=data))
            except Exception as e:
                results.append(Result(task_id=task.id, success=False, error=str(e)))

    # If cancelled by deadline, count remaining
    if scope.cancel_called:
        skipped = len(tasks) - len(results)

    return results, skipped


# Demo: Process 20 tasks with 1 second deadline
async def work(task_id: str, duration: float):
    await sleep(duration)
    return f"Result-{task_id}"


tasks = [Task(id=f"t{i}", work=lambda i=i: work(f"t{i}", 0.1)) for i in range(20)]
deadline = current_time() + 1.0

results, skipped = await process_sequential_with_deadline(tasks, deadline)

print(f"Completed: {len([r for r in results if r.success])}/{len(tasks)}")
print(f"Failed: {len([r for r in results if not r.success])}")
print(f"Skipped: {skipped}")
print(f"\nFirst 3 results: {[r.data for r in results[:3]]}")

Completed: 9/20
Failed: 0
Skipped: 11

First 3 results: ['Result-t0', 'Result-t1', 'Result-t2']


## Sequential Pattern Explanation

**Key mechanics**:
1. `move_on_at(deadline)` stops execution at deadline
2. Before each task: check `remaining = deadline - current_time()`
3. If `remaining < min_time`, stop (not enough time)
4. `scope.cancel_called` indicates deadline was reached

**Why check time explicitly?** Prevents starting tasks you can't finish (wasted CPU/memory).

**Use cases**: Rate-limited APIs, database transactions, sequential file I/O.

## Pattern 2: Parallel Worker Pool with Queue

**When to use**: Tasks are independent and can run concurrently (I/O-bound work, parallel API calls)

**Key insight**: Multiple workers pull from shared queue, each checking deadline before processing.

**Pattern**:
```
queue = Queue.with_maxsize(100)
async with TaskGroup() as tg:
    for i in range(num_workers):
        tg.create_task(worker(i, queue))
    
    for task in tasks:
        await queue.put(task)
    await queue.put(SENTINEL)  # Signal shutdown
```

In [3]:
# Parallel worker pool with deadline awareness

SENTINEL = object()  # Shutdown signal


async def deadline_aware_worker(
    worker_id: int,
    queue: Queue,
    results: list[Result],
    min_time: float = 0.01,
) -> None:
    """Worker that processes items from queue until deadline.

    Args:
        worker_id: Unique worker identifier
        queue: Shared work queue
        results: Shared results list (thread-safe due to GIL)
        min_time: Skip if less time remaining
    """
    while True:
        item = await queue.get()

        # Check for sentinel (graceful shutdown)
        if item is SENTINEL:
            await queue.put(SENTINEL)  # Propagate to other workers
            break

        # Check remaining time
        deadline = effective_deadline()
        if deadline and (deadline - current_time()) < min_time:
            # Not enough time, skip this task
            continue

        # Process the task
        task: Task = item
        try:
            data = await task.work()
            results.append(Result(task_id=task.id, success=True, data=data))
        except Exception as e:
            results.append(Result(task_id=task.id, success=False, error=str(e)))


async def process_parallel_with_deadline(
    tasks: list[Task],
    deadline: float,
    num_workers: int = 4,
    queue_size: int = 100,
) -> list[Result]:
    """Process tasks in parallel with multiple workers.

    Args:
        tasks: List of tasks to process
        deadline: Absolute deadline
        num_workers: Number of concurrent workers
        queue_size: Maximum queue size (backpressure limit)

    Returns:
        List of results from all workers
    """
    queue = Queue.with_maxsize(queue_size)
    results = []

    async def producer():
        """Feed tasks into queue."""
        for task in tasks:
            await queue.put(task)
        await queue.put(SENTINEL)  # Signal workers to stop

    with move_on_at(deadline):
        async with create_task_group() as tg:
            # Spawn workers
            for i in range(num_workers):
                tg.start_soon(deadline_aware_worker, i, queue, results)

            # Feed queue
            tg.start_soon(producer)

    return results


# Demo: Process 50 tasks with 4 workers and 1 second deadline
async def parallel_work(task_id: str, duration: float):
    await sleep(duration)
    return f"Result-{task_id}"


tasks = [Task(id=f"t{i}", work=lambda i=i: parallel_work(f"t{i}", 0.05)) for i in range(50)]
deadline = current_time() + 1.0

start = current_time()
results = await process_parallel_with_deadline(tasks, deadline, num_workers=4)
elapsed = current_time() - start

successful = len([r for r in results if r.success])
print(f"Completed: {successful}/{len(tasks)} tasks")
print(f"Total time: {elapsed:.2f}s")
print(f"Expected sequential time: ~{len(tasks) * 0.05:.2f}s")
print(f"Speedup: ~{(len(tasks) * 0.05) / elapsed:.1f}x")

Completed: 50/50 tasks
Total time: 0.67s
Expected sequential time: ~2.50s
Speedup: ~3.8x


## Worker Pool Pattern Explanation

**Key mechanics**:
1. Bounded queue (`Queue.with_maxsize(n)`) controls memory
2. `num_workers` tasks pull from shared queue
3. Producer feeds tasks, then puts `SENTINEL` to signal shutdown
4. Workers check `effective_deadline()` before processing

**Sentinel pattern**: Each worker checks for `SENTINEL` → re-queues it → exits. This ensures all workers see the shutdown signal.

**Bounded queue benefits**: Prevents memory exhaustion, provides backpressure (producer blocks when full).

**Use cases**: Batch API requests, data transformations, parallel database queries.

In [4]:
# Sentinel pattern demonstration


async def demonstrate_sentinel():
    """Show how sentinel propagates through workers."""
    queue = Queue.with_maxsize(10)
    worker_exits = []

    async def tracking_worker(worker_id: int):
        """Worker that tracks when it exits."""
        items_processed = 0
        while True:
            item = await queue.get()
            if item is SENTINEL:
                await queue.put(SENTINEL)  # Critical: re-queue!
                worker_exits.append((worker_id, items_processed))
                break
            items_processed += 1
            await sleep(0.01)

    # Spawn 3 workers
    async with create_task_group() as tg:
        for i in range(3):
            tg.start_soon(tracking_worker, i)

        # Feed 5 items then sentinel
        for i in range(5):
            await queue.put(f"item-{i}")
        await queue.put(SENTINEL)

    print("Worker exits (worker_id, items_processed):")
    for worker_id, count in sorted(worker_exits):
        print(f"  Worker {worker_id}: processed {count} items")
    print(f"\nTotal items processed: {sum(c for _, c in worker_exits)}")


await demonstrate_sentinel()

Worker exits (worker_id, items_processed):
  Worker 0: processed 2 items
  Worker 1: processed 2 items
  Worker 2: processed 1 items

Total items processed: 5


## Comparison: Sequential vs Parallel

| Aspect | Sequential | Parallel (Worker Pool) |
|--------|-----------|------------------------|
| **Throughput** | 1x (baseline) | N× (N = num_workers, for I/O-bound) |
| **Complexity** | Simple (single loop) | Moderate (queue, workers, sentinel) |
| **Memory** | Low (one task at a time) | Medium (queue + N active tasks) |
| **Order** | Preserved | Not preserved (results unordered) |
| **Error Isolation** | Single failure visible | Per-worker isolation |
| **Best For** | Rate-limited APIs, sequential deps | Independent I/O-bound tasks |

**Decision guide**:

**Use Sequential when**:
- Tasks have dependencies (must run in order)
- Rate limits require delays between tasks
- Single resource (file handle, DB connection)
- Task count < 100 and deadline is generous

**Use Parallel when**:
- Tasks are independent (no shared state)
- I/O-bound work (network, disk, database)
- Task count > 100 or tight deadline
- Have spare CPU/memory for workers

**Avoid Parallel when**:
- CPU-bound tasks on single-core (GIL limits parallelism)
- Memory-constrained (workers + queue = high memory)
- Tasks have complex dependencies (use DAG instead)

## Summary

**Key APIs**: `move_on_at()`, `effective_deadline()`, `current_time()`, `Queue.with_maxsize()`

**Patterns**:
- Sequential: Check time before each task
- Parallel: Worker pool with shared queue + sentinel shutdown

**When to use**:
- ✅ Background jobs with deadlines (ETL, reports, batch notifications)
- ✅ Maintenance windows with fixed time slots
- ❌ Real-time where every task must complete (use per-task timeouts)
- ❌ CPU-bound on single-core (GIL limits parallel speedup)

**Related**: [Cancellation API](../../docs/api/libs/concurrency/cancel.md), [Primitives API](../../docs/api/libs/concurrency/primitives.md)

## Common Patterns and Variations

### 1. Per-Task Timeout within Overall Deadline

Sometimes you need both: individual task timeout + overall deadline.

```python
from lionherd_core.libs.concurrency import move_on_after

with move_on_at(deadline):  # Overall deadline
    for task in tasks:
        with move_on_after(task_timeout):  # Per-task timeout
            result = await task()
        # Task timed out if scope.cancel_called
```

### 2. Priority Queue for Important Tasks First

Process high-priority tasks before deadline hits.

```python
from lionherd_core.libs.concurrency import PriorityQueue

queue = PriorityQueue.with_maxsize(100)

# Put with priority (lower number = higher priority)
await queue.put((priority, task))

# Worker pulls highest priority first
priority, task = await queue.get()
```

### 3. Adaptive Worker Count Based on Deadline

Scale workers based on remaining time.

```python
remaining = deadline - current_time()
tasks_left = len(tasks)
avg_task_time = 0.05  # Estimated

# Calculate needed workers
needed_workers = int((tasks_left * avg_task_time) / remaining) + 1
num_workers = min(needed_workers, max_workers)
```

### 4. Checkpoint and Resume

Save progress periodically for long-running jobs.

```python
checkpoint_interval = 60.0  # Save every 60s
last_checkpoint = current_time()

for task in tasks:
    result = await task()
    results.append(result)
    
    if current_time() - last_checkpoint > checkpoint_interval:
        save_checkpoint(results)
        last_checkpoint = current_time()
```

## Summary and Next Steps

**What You Learned**:
- Sequential deadline processing: check time before each task
- Parallel worker pool: shared queue + multiple workers + sentinel shutdown
- Core APIs: `move_on_at()`, `effective_deadline()`, `current_time()`, `Queue.with_maxsize()`
- When to use sequential vs parallel patterns

**Key Takeaways**:
1. **Absolute deadlines** (`move_on_at`) are better than per-task timeouts for batch work
2. **Check time before starting** tasks to avoid wasted work
3. **Sentinel pattern** enables graceful shutdown (finish current task, then exit)
4. **Bounded queues** prevent memory exhaustion in worker pools
5. **Parallel is not always faster** - only helps for I/O-bound independent tasks

**When to Use Deadline Patterns**:
- ✅ Background jobs with completion deadlines (ETL, reports, batch notifications)
- ✅ Maintenance windows with fixed time slots
- ✅ Rate-limited API operations (maximize throughput within budget)
- ❌ Real-time processing where every task must complete (use per-task timeouts)
- ❌ CPU-bound tasks on single-core (GIL limits parallel speedup)

**Related Resources**:
- [API Reference: Cancellation Utilities](../../docs/api/libs/concurrency/cancel.md)
- [API Reference: Primitives](../../docs/api/libs/concurrency/primitives.md)
- [Reference Notebook: Concurrency Primitives](../references/concurrency_primitives.ipynb)
- [Reference Notebook: Cancellation](../references/concurrency_cancel.ipynb)

**Next Tutorials**:
- Circuit Breaker Pattern (resilient service calls)
- Retry Strategies (exponential backoff, jitter)
- Rate Limiting (token bucket, leaky bucket)